In [371]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [368]:
def parse_patent(html_code):
    """Функция собирает всю необходимую информацию о патенте с html-кода.

    Args:
        html_code (lxml): html-код страницы патента 

    Returns:
        list: [
                date - дата подачи заявки,
                quotes - список цитированных в отчёте документов,
                authors - авторы,
                patent_owner - патентообладатель,
                mpk - МПК,
                spk - СПК,
                country - страна,
                number - Номер патента,
                type_of_document - тип документа,
                title - название патента,
                abstract - реферат,
                sources_of_information - источники информации,
                patent_claims - формула патента,
                patent_description - описание патента,
                ]

    """
    keys_data_1 = html_code.find('table', id='bib').find('tr').findAll('td')[0].findAll('p')
    keys_data_2 = html_code.find('table', id='bib').find('tr').findAll('td')[1].findAll('p')

    ind_of_prior = [ind for ind, i in enumerate(keys_data_1) if 'Приоритет' in i.text][0] # индекс "Приоритет(ы)"
    date = keys_data_1[ind_of_prior+1].text.split()[-1] # Дата подачи заявки
    quotes = keys_data_1[ind_of_prior+3].find('b').text.split('. ') # Список документов, цитированных в отчете о поиске 5 или 6?
    
    authors = keys_data_2[0].find('b').text[1:].split(',') # Автор(ы)
    patent_owner = keys_data_2[1].find('b').text[1:] # Патентообладатель(и)

    keys_data_3 = html_code.find('table', class_='tp').findAll('tr')
    mpk = [' '.join(i.text[1:-1].split()) for i in keys_data_3[3].find('div').find('ul').findAll('li')] # МПК
    try:
        spk = [' '.join(i.text[1:-1].split()) for i in keys_data_3[5].find('div').find('ul').findAll('li')] # СПК
    except AttributeError:
        spk = [keys_data_3[5].text]

    keys_data_4 = html_code.find('table', class_='tp').findAll('div', class_='topfield2')
    country = keys_data_4[0].text # Страна
    number = keys_data_4[1].text[1:-1].replace(" ", "") # Номер патента
    type_of_document = keys_data_4[2].text # Тип документа

    title = ' '.join(html_code.find('p', id='B542').text.split()[1:]) # Название патента
    abstract = html_code.find('div', id='Abs').findAll('p')[1].text[:-1] # Реферат патента

    all_patent = [i.text for i in html_code.findAll('p')[13:-8] if not ('\n\n' == i.text)] # Весь патент

    ind_source_of_information = [ind for ind, i in enumerate(all_patent) if i == "Источники информации"][0] # Индекс "Источники информации"
    ind_patent_claims = [ind for ind, i in enumerate(all_patent) if i == "Формула изобретения"][0] # индекс "Формула патента"

    patent_description = all_patent[:ind_source_of_information] # Описание патента без реферата и формулы
    sources_of_information = all_patent[ind_source_of_information+1:ind_patent_claims] # Источники информации
    patent_claims = all_patent[ind_patent_claims+1:] # Формула патента

    return [date, quotes, authors, patent_owner, mpk, spk, country, number, type_of_document, title, abstract, sources_of_information, patent_claims, patent_description]


In [369]:
urls = ["https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2637505&TypeFile=html",
"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2640290&TypeFile=html"]

infos_about_patents = []
for link in urls:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    parsed_info = parse_patent(soup)
    infos_about_patents.append(parsed_info)

In [370]:
headers = ['date', 'quotes', 'authors', 'patent_owner', 'mpk', 'spk', 'country', 'number', 'type_of_document', 'title', 'abstract', 'sources_of_information', 'patent_claims', 'patent_description']
infos = pd.DataFrame(infos_about_patents, columns=headers)
infos.to_csv('about_patents1.csv')